<a href="https://colab.research.google.com/github/alevin0521/0123-linebot/blob/main/1201lol_bot_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# loltestbot功能
# 第一步掛載Google硬碟
# 第二步安裝套件
# 第三步引用套件
# 第四步創建好Server跟物件
# 第五步開啟接口準備接收line消息，跟line的驗證機制
# 第六步用將消息內容存在google drive下的檔案內 ，用handler進行消息驗證

# 上面都一樣
# 注意按鍵範本最多4個

#app啟動

In [ ]:
# 資料 mapping 至google drive
# 把資料寫在/content/drive
# 即可保存在 google drive內

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 安裝套件
!pip install line-bot-sdk flask flask-ngrok

     |████████████████████████████████| 71kB 3.2MB/s 


In [ ]:
# 引用套件

# 引用Web Server套件
from flask import Flask, request, abort, jsonify

# 載入json處理套件
import json

# 外部連結自動生成套件
from flask_ngrok import run_with_ngrok

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)

In [ ]:
# 建置主程序app
# 建置handler與 line_bot_api

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/material" , static_folder = "./material/")
run_with_ngrok(app)

# 生成實體物件
line_bot_api = LineBotApi("Your channel access token")
#channel access token
handler = WebhookHandler("Your Channel secret")   
#Channel secret

In [ ]:
# 建置主程序的API入口
#   接受Line傳過來的消息
#   並取出消息內容
#   將消息內容存在google drive的檔案內
#   並請handler 進行消息驗證與轉發


#001設定method是GET，003是POST:啟動server對外接口，使Line能丟消息進來  
@app.route("/", methods=['POST'])
def callback():
    #如果該HTTP POST訊息是來自LINE平台，在HTTP請求標頭中一定會包括X-Line-Signature項目
    # request.headers:拿取HTTP請求的標頭內容，拿X-Line-Signature標頭裡的
    #內容裡放數位簽章 避免被偽造 
    signature = request.headers['X-Line-Signature']
   
    # get request body as text
    #Flask後臺可使用 request.get_data() 來接收數據，數據的格式為 bytes
    #加上as_text=True 參數後就變成 Unicode 了
    body = request.get_data(as_text=True)
    print(body)

    # 用OS模組中寫檔案功能  記錄用戶log 
    f = open("/content/drive/MyDrive/ai-event.log", "a")
    f.write(body)
    f.close()

    # handler.handle(body, signature): 會幫你判斷要用剛剛設定的哪個 event 來作回應
    # InvalidSignatureError: Line sdk 會檢查你的資料是否正確

    # handle webhook body
    try:
      # handler驗證消息
        handler.handle(body, signature)
    except InvalidSignatureError:
      # 若驗證失敗，就退回，回傳代碼400
        abort(400)

    return 'OK'

In [ ]:
#下面做lol bot 的功能
#第一步 第一次加入機器人時 觸發關注事件  發送歡迎訊息
#第二步 輸入@裝備名字 有裝備圖跑出來  建置其字典與圖片庫
#第三步 輸入@實況主 跳出按鍵範本可以點選
#第四步 輸入@攻略 有網址跳出來
#第五步 可以成功傳送 貼圖


In [ ]:
# 觸發關注事件時 要執行的內容    #第一步 第一次加入機器人時 觸發關注事件  發送歡迎訊息

# 1. 取得用戶個資，並存回伺服器
# 2. 回應用戶，歡迎用的文字消息   

# 載入Follow事件
from linebot.models.events import (
    FollowEvent
)

# 將圖片、文字寄發模組 引入
from linebot.models import(
    TextSendMessage, ImageSendMessage
)

# 若handler 收到關注事件時 ，做下面的事情
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    
    # 取出消息內User的資料
    user_profile = line_bot_api.get_profile(event.source.user_id)
        
     # 將用戶資訊存在檔案內
     # python svae file
    with open("/content/drive/MyDrive/users.txt", "a") as myfile:
        myfile.write(json.dumps(vars(user_profile),sort_keys=True))
        myfile.write('\n')
    
    # 回覆文字消息與圖片消息  不知道是不是只能回一種訊息
    line_bot_api.reply_message(
        event.reply_token,
        [TextSendMessage('用戶你好，歡迎使用此機器人，可以點擊下方選單或輸入@裝備名字、@攻略、@實況主、@安安來啟用機器人')]
        
    )

In [ ]:
# 將訊息事件、文字訊息、寄送文字與圖片 模組引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)

#甚麼事件都沒有  直接寄送文字訊息 
send_text_message=TextSendMessage("官方建議，用戶以文字消息觸發特殊功能的時候，要加上特殊符號@")
#寄送圖片訊息  1001到1054
send_image_message1001=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1001.png',  #鞋子1001
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1001.png'    #鞋子1001
      )
send_image_message1004=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1004.png',  #仙女護符1004
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1004.png'    #仙女護符1004
      )
send_image_message1006=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1006.png',  #治療寶珠1006
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1006.png'    #治療寶珠1006
      )
send_image_message1011=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1011.png',  #巨人腰帶1011                        
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1011.png'    #巨人腰帶1011
      )
send_image_message1018=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1018.png',  #靈巧披風1018
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1018.png'    #靈巧披風1018
      )
send_image_message1026=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1026.png',  #爆裂魔杖1026
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1026.png'    #爆裂魔杖1026
      )
send_image_message1027=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1027.png',  #藍水晶1027
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1027.png'    #藍水晶1027
      )
send_image_message1028=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1028.png',  #紅水晶1028
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1028.png'    #紅水晶1028
      )
send_image_message1029=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1029.png',  #布甲1029
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1029.png'    #布甲1029
      )
send_image_message1031=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1031.png',  #鎖子甲1031
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1031.png'    #鎖子甲1031
      )
send_image_message1033=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1033.png',  #抗魔斗篷1033
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1033.png'    #抗魔斗篷1033
      )
send_image_message1035=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1035.png',  #餘燼之刃1035
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1035.png'    #餘燼之刃1035
      )
send_image_message1036=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1036.png',  #長劍1036
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1036.png'    #長劍1036
      )
send_image_message1037=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1037.png',  #十字鎬1037
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1037.png'    #十字鎬1037
      )
send_image_message1038=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1038.png',  #暴風大劍1038
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1038.png'    #暴風大劍1038
      )
send_image_message1039=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1039.png',  #雹刃1039
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1039.png'    #雹刃1039
      )
send_image_message1042=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1042.png',  #短劍1042
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1042.png'    #短劍1042
      )
send_image_message1043=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1043.png',  #反曲弓1043
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1043.png'    #反曲弓1043
      )
send_image_message1052=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1052.png',  #增幅典籍1052
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1052.png'    #增幅典籍1052
      )
send_image_message1053=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1053.png',  #吸血鬼權杖1053
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1053.png'    #吸血鬼權杖1053
      )
send_image_message1054=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1054.png',  #多蘭之盾1054
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1054.png'    #多蘭之盾1054
      )


In [ ]:
#發圖片1055-3036
send_image_message1055=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1055.png',  #多蘭之劍1055
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1055.png'    #多蘭之劍1055
      )
send_image_message1056=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1056.png',  #多蘭之戒1056
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1056.png'    #多蘭之戒1056
      )
send_image_message1057=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1057.png',  #負極斗篷1057
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1057.png'    #負極斗篷1057
      )
send_image_message1058=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1058.png',  #巨型魔棒1058
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1058.png'    #巨型魔棒1058
      )
send_image_message1082=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1082.png',  #黑暗玉璽1082
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1082.png'    #黑暗玉璽1082
      )
send_image_message1083=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1083.png',  #汰除品1083
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/1083.png'    #汰除品1083
      )
send_image_message2003=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/2003.png',  #生命藥水2003
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/2003.png'    #生命藥水2003
      )
send_image_message2015=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/2015.png',  #科奇碎片2015
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/2015.png'    #科奇碎片2015
      )
send_image_message2031=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/2031.png',  #回復藥水2031
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/2031.png'    #回復藥水2031
      )
send_image_message2033=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/2033.png',  #汙濁藥劑2033
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/2033.png'    #汙濁藥劑2033
      )
send_image_message2055=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/2055.png',  #控域守衛2055
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/2055.png'    #控域守衛2055
      )
send_image_message2065=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/2065.png',  #蘇瑞雅的戰歌2065
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/2065.png'    #蘇瑞雅的戰歌2065
      )
send_image_message2138=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/2138.png',  #抗擊藥劑2138
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/2138.png'    #抗擊藥劑2138
      )
send_image_message2139=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/2139.png',  #巫魔藥劑2139
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/2139.png'    #巫魔藥劑2139
      )
send_image_message2140=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/2140.png',  #憤怒藥劑2140
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/2140.png'    #憤怒藥劑2140
      )
send_image_message2419=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/2419.png',  #預備鐘錶2419
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/2419.png'    #預備鐘錶2419
      )
send_image_message2420=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/2420.png',  #停滯鐘錶2420
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/2420.png'    #停滯鐘錶2420
      )
send_image_message2422=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/2422.png',  #輕魔之靴2422
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/2422.png'    #輕魔之靴2422
      )
send_image_message2423=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/2423.png',  #完美校準的停滯鐘錶2423
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/2423.png'    #完美校準的停滯鐘錶2423
      )
send_image_message3001=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3001.png',  #深淵面具3001
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3001.png'    #深淵面具3001
      )
send_image_message3003=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3003.png',  #大天使之杖3003
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3003.png'    #大天使之杖3003
      )
send_image_message3004=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3004.png',  #魔劍正宗3004
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3004.png'    #魔劍正宗3004
      )
send_image_message3006=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3006.png',  #狂戰士護脛3006
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3006.png'    #狂戰士護脛3006
      )
send_image_message3009=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3009.png',  #輕靈之靴3009
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3009.png'    #輕靈之靴3009
      )
send_image_message3011=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3011.png',  #生化科技淨化器3011
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3011.png'    #生化科技淨化器3011
      )
send_image_message3020=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3020.png',  #法師之靴3020
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3020.png'    #法師之靴3020
      )
send_image_message3024=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3024.png',  #冰川圓盾3024
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3024.png'    #冰川圓盾3024
      )
send_image_message3026=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3026.png',  #守護天使3026
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3026.png'    #守護天使3026
      )
send_image_message3031=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3031.png',  #無盡之刃3031
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3031.png'    #無盡之刃3031
      )
send_image_message3033=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3033.png',  #致死宣告3033
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3033.png'    #致死宣告3033
      )
send_image_message3035=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3035.png',  #最後耳語3035
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3035.png'    #最後耳語3035
      )
send_image_message3036=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3036.png',  #多明尼克的問候3036
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3036.png'    #多明尼克的問候3036
      )


In [ ]:
#傳圖片3040-30
#3042跟3043 一樣圖 做一個就好  3040跟3048
send_image_message3040=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3040.png',  #熾天使的擁抱3040
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3040.png'    #熾天使的擁抱3040
      )
send_image_message3041=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3041.png',  #靈魂竊取者3041
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3041.png'    #靈魂竊取者3041
      )
send_image_message3042=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3042.png',  #終極魔劍3042
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3042.png'    #終極魔劍3042
      )
send_image_message3044=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3044.png',  #淨蝕之槌3044
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3044.png'    #淨蝕之槌3044
      )
send_image_message3046=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3046.png',  #幻影之舞3046
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3046.png'    #幻影之舞3046
      )
send_image_message3047=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3047.png',  #鍍板鋼蓋3047
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3047.png'    #鍍板鋼蓋3047
      )
send_image_message3050=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3047.png',  #錫柯的聚合之力3050
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3047.png'    #錫柯的聚合之力3050
      )
send_image_message3051=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3047.png',  #爐縛之斧3051
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3047.png'    #爐縛之斧3051
      )
send_image_message3053=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3047.png',  #史特拉克手套3053
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3047.png'    #史特拉克手套3053
      )
send_image_message3057=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3057.png',  #閃耀劍3057
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3057.png'    #閃耀劍3057
      )
send_image_message3065=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3065.png',  #振奮盔甲3065
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3065.png'    #振奮盔甲3065
      )
send_image_message3066=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3066.png',  #飛翼月甲3066
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3066.png'    #飛翼月甲3066
      )
send_image_message3067=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3067.png',  #燃燒寶石3067
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3067.png'    #燃燒寶石3067
      )
send_image_message3068=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3068.png',  #日炎聖盾3068
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3068.png'    #日炎聖盾3068
      )
send_image_message3070=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3070.png',  #女神之淚3070
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3070.png'    #女神之淚3070
      )
send_image_message3071=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3071.png',  #黑色切割者3071
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3071.png'    #黑色切割者3071
      )
send_image_message3072=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3072.png',  #嗜血者3072
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3072.png'    #嗜血者3072
      )
send_image_message3074=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3074.png',  #狂怒九頭蛇3074
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3074.png'    #狂怒九頭蛇3074
      )
send_image_message3075=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3075.png',  #荊棘之甲3075
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3075.png'    #荊棘之甲3075
      )
send_image_message3076=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3076.png',  #刺藤胸甲3076
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3076.png'    #刺藤胸甲3076
      )
send_image_message3077=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3077.png',  #海神之斧3077
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3077.png'    #海神之斧3077
      )
send_image_message3078=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3078.png',  #三相之力3078
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3078.png'    #三相之力3078
      )
send_image_message3082=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3082.png',  #守望者鎧甲3082
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3082.png'    #守望者鎧甲3082
      )
send_image_message3083=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3083.png',  #好戰者鎧甲3083
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3083.png'    #好戰者鎧甲3083
      )
send_image_message3085=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3085.png',  #芮蘭颶風箭3085
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3085.png'    #芮蘭颶風箭3085
      )
send_image_message3086=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3086.png',  #狂熱雙刀3086
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3086.png'    #狂熱雙刀3086
      )
send_image_message3089=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3089.png',  #死亡之帽3089
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3089.png'    #死亡之帽3089
      )
send_image_message3091=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3091.png',  #智慧末刃3091
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3091.png'    #智慧末刃3091
      )
send_image_message3094=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3094.png',  #衝擊火炮3094
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3094.png'    #衝擊火炮3094
      )
send_image_message3095=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3095.png',  #狂暴利刃3095
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3095.png'    #狂暴利刃3095
      )
send_image_message3100=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3100.png',  #巫妖之禍3100
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3100.png'    #巫妖之禍3100
      )
send_image_message3102=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3102.png',  #女妖面紗3012
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3102.png'    #女妖面紗3012
      )


In [ ]:
#傳圖片 3105--6695  
#3112 沒有保衛者冰玉  #海克斯豪飲??吸魔劍3155  保衛者戰鎚3184 沒有 3330稻草人
#3340 潛行守衛 3363 鷹眼晶球 3364 先知透視鏡
                                                                       
send_image_message3105=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3105.png',  #軍團聖盾3105
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3105.png'    #軍團聖盾3105
      )
send_image_message3107=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3107.png',  #贖罪神石3107
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3107.png'    #贖罪神石3107
      )
send_image_message3108=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3108.png',  #惡魔法典3108
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3108.png'    #惡魔法典3108
      )
send_image_message3109=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3109.png',  #騎士誓願3109
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3109.png'    #騎士誓願3109
      )
send_image_message3110=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3110.png',  #冰霜之心3110
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3110.png'    #冰霜之心3110
      )
send_image_message3111=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3111.png',  #水星之靴3111
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3111.png'    #水星之靴3111
      )
send_image_message3113=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3113.png',  #蒼穹神火3113
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3113.png'    #蒼穹神火3113
      )
send_image_message3114=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3114.png',  #禁忌之像3114
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3114.png'    #禁忌之像3114
      )
send_image_message3115=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3115.png',  #納什之牙3115
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3115.png'    #納什之牙3115
      )
send_image_message3116=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3116.png',  #瑞萊的冰晶節杖3116
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3116.png'    #瑞萊的冰晶節杖3116
      )
send_image_message3117=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3117.png',  #疾行之靴3117
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3117.png'    #疾行之靴3117
      )
send_image_message3123=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3123.png',  #死刑宣告3123
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3123.png'    #死刑宣告3123
      )
send_image_message3124=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3078.png',  #鬼索的狂暴之刃3124
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3078.png'    #鬼索的狂暴之刃3124
      )
send_image_message3133=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3133.png',  #考菲爾德的戰槌3133
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3133.png'    #考菲爾德的戰槌3133
      )
send_image_message3134=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3134.png',  #殘暴之力3134
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3134.png'    #殘暴之力3134
      )
send_image_message3135=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3135.png',  #虛空之杖3135
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3135.png'    #虛空之杖3135
      )
send_image_message3139=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3139.png',  #水星彎刀3139
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3139.png'    #水星彎刀3139
      )
send_image_message3140=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3140.png',  #水銀飾帶3140
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3140.png'    #水銀飾帶3140
      )
send_image_message3142=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3142.png',  #妖夢鬼刀3142
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3142.png'    #妖夢鬼刀3142
      )
send_image_message3143=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3143.png',  #蘭頓之兆3143
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3143.png'    #蘭頓之兆3143
      )
send_image_message3145=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3145.png',  #海克斯科技發電機3145
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3145.png'    #海克斯科技發電機3145
      )
send_image_message3152=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3152.png',  #海克斯科技火箭腰帶3152
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3152.png'    #海克斯科技火箭腰帶3152
      )
send_image_message3153=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3153.png',  #殞落王者之劍3153
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3153.png'    #殞落王者之劍3153
      )
send_image_message3155=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3155.png',  #吸魔劍3155
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3155.png'    #吸魔劍3155
      )
send_image_message3156=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3156.png',  #魔提斯深淵3156
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3156.png'    #魔提斯深淵3156
      )
send_image_message3157=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3157.png',  #中婭沙漏3157
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3157.png'    #中婭沙漏3157
      )
send_image_message3158=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3158.png',  #愛歐尼亞之靴3158
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3158.png'    #愛歐尼亞之靴3158
      )
send_image_message3165=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3165.png',  #黑魔禁書3165
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3165.png'    #黑魔禁書3165
      )
send_image_message3177=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3177.png',  #守護者之刃3177
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3177.png'    #守護者之刃3177
      )
send_image_message3179=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3179.png',  #闇影戰戟3179
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3179.png'    #闇影戰戟3179
      )
send_image_message3181=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3181.png',  #愉悅刀刃3181
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3181.png'    #愉悅刀刃3181
      )
send_image_message3190=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3190.png',  #日輪的加冕3190
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3190.png'    #日輪的加冕3190
      )
send_image_message3191=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3191.png',  #探索者護腕3191
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3191.png'    #探索者護腕3191
      )
send_image_message3193=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3193.png',  #石像鬼磐核3193
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3193.png'    #石像鬼磐核3193
      )
send_image_message3211=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3211.png',  #亡靈斗篷3211
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3211.png'    #亡靈斗篷3211
      )
send_image_message3222=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3222.png',  #米凱的祝福3222
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3222.png'    #米凱的祝福3222
      )
send_image_message3504=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3504.png',  #熾灼魔器3504
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3504.png'    #熾灼魔器3504
      )
send_image_message3508=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3508.png',  #奪魄之鐮3508
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3508.png'    #奪魄之鐮3508
      )
send_image_message3742=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3742.png',  #死者肩甲3742
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3742.png'    #死者肩甲3742
      )
send_image_message3748=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3748.png',  #泰坦九頭蛇3748
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3748.png'    #泰坦九頭蛇3748
      )
send_image_message3801=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3801.png',  #水晶護腕3801
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3801.png'    #水晶護腕3801
      )
send_image_message3802=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3802.png',  #失落之章3802
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3802.png'    #失落之章3802
      )
send_image_message3814=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3814.png',  #夜色緣界3814
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3814.png'    #夜色緣界3814
      )
send_image_message3850=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3850.png',  #盜竊之刃3850
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3850.png'    #盜竊之刃3850
      )
send_image_message3851=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3851.png',  #霜牙3851
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3851.png'    #霜牙3851
      )
send_image_message3853=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3853.png',  #真霜碎片3853
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3853.png'    #真霜碎片3853
      )
send_image_message3854=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3854.png',  #鋼鐵護肩3854
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3854.png'    #鋼鐵護肩3854
      )
send_image_message3855=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3855.png',  #符文鋼肩3855
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3855.png'    #符文鋼肩3855
      )
send_image_message3857=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3857.png',  #白石韌肩3857
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3857.png'    #白石韌肩3857
      )
send_image_message3858=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3858.png',  #遺跡之盾3858
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3858.png'    #遺跡之盾3858
      )
send_image_message3859=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3859.png',  #巨石峰圓盾3859
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3859.png'    #巨石峰圓盾3859
      )
send_image_message3860=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3860.png',  #群山堡壘3860
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3860.png'    #群山堡壘3860
      )
send_image_message3862=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3862.png',  #幽魂之鐮3862
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3862.png'    #幽魂之鐮3862
      )
send_image_message3863=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3863.png',  #駭懼之鐮3863
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3863.png'    #駭懼之鐮3863
      )
send_image_message3864=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3864.png',  #黯霧之鐮3864
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3864.png'    #黯霧之鐮3864
      )
send_image_message3916=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3916.png',  #遺忘寶珠3916
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/3916.png'    #遺忘寶珠3916
      )
send_image_message4005=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/4005.png',  #帝王命令4005
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/4005.png'    #帝王命令4005
      )
send_image_message4401=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/4401.png',  #自然之力4401
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/4401.png'    #自然之力4401
      )
send_image_message4628=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/4628.png',  #視界專注4628
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/4628.png'    #視界專注4628
      )
send_image_message4629=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/4629.png',  #星河馳騁4629
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/4629.png'    #星河馳騁4629
      )
send_image_message4630=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/4630.png',  #枯萎寶石4630
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/4630.png'    #枯萎寶石4630
      )
send_image_message4632=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/4632.png',  #蒼翠屏障4632
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/4632.png'    #蒼翠屏障4632
      )
send_image_message4633=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/4633.png',  #峽谷製造者4633
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/4633.png'    #峽谷製造者4633
      )
send_image_message4635=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/4635.png',  #榨血睥睨4635
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/4635.png'    #榨血睥睨4635
      )
send_image_message4636=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/4636.png',  #闇夜收割者4636
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/4636.png'    #闇夜收割者4636
      )
send_image_message4637=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/4637.png',  #惡魔之擁4637
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/4637.png'    #惡魔之擁4637
      )
send_image_message4638=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/4638.png',  #警戒視石4638
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/4638.png'    #警戒視石4638
      )
send_image_message4641=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/4641.png',  #活耀視石4641
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/4641.png'    #活耀視石4641
      )
send_image_message4642=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/4642.png',  #班德爾玻璃之鏡4642
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/4642.png'    #班德爾玻璃之鏡4642
      )
send_image_message4643=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/4643.png',  #警覺視石4643
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/4643.png'    #警覺視石4643
      )
send_image_message6029=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6029.png',  #鐵刺之鞭6029
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6029.png'    #鐵刺之鞭6029
      )
send_image_message6035=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6035.png',  #銀織黎明6035
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6035.png'    #銀織黎明6035
      )
send_image_message6333=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6333.png',  #死亡之舞6333
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6333.png'    #死亡之舞6333
      )
send_image_message6609=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6609.png',  #生化龐克鏈鋸之劍6609
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6609.png'    #生化龐克鏈鋸之劍6609
      )
send_image_message6616=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6616.png',  #流水之杖6616
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6616.png'    #流水之杖6616
      )
send_image_message6617=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6617.png',  #月之石再生裝置6617
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6617.png'    #月之石再生裝置6617
      )
send_image_message6630=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6630.png',  #飲血戰錘6630
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6630.png'    #飲血戰錘6630
      )
send_image_message6631=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6631.png',  #步履破壞者6631
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6631.png'    #步履破壞者6631
      )
send_image_message6632=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6632.png',  #聖裂之杖6632
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6632.png'    #聖裂之杖6632
      )
send_image_message6653=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6653.png',  #黎安卓的苦楚6653
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6653.png'    #黎安卓的苦楚6653
      )
send_image_message6655=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6655.png',  #盧登驟雨6655
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6655.png'    #盧登驟雨6655
      )
send_image_message6656=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6656.png',  #永寒之霜6656
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6656.png'    #永寒之霜6656
      )
send_image_message6660=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6660.png',  #巴米灰燼6660
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6660.png'    #巴米灰燼6660
      )
send_image_message6662=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6662.png',  #霜火霸拳6662
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6662.png'    #霜火霸拳6662
      )
send_image_message6664=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6664.png',  #渦輪煉金罐6664
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6664.png'    #渦輪煉金罐劍6664
      )
send_image_message6670=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6670.png',  #正午之顫6670
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6670.png'    #正午之顫6670
      )
send_image_message6671=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6671.png',  #狂風之力6671
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6671.png'    #狂風之力6671
      )
send_image_message6672=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6672.png',  #海妖殺手6672
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6672.png'    #海妖殺手6672
      )
send_image_message6673=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6673.png',  #不朽盾弓6673
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6673.png'    #不朽盾弓6673
      )
send_image_message6675=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6675.png',  #那歐疾刃6675
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6675.png'    #那歐疾刃6675
      )
send_image_message6676=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6676.png',  #蒐集者6676
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6676.png'    #蒐集者6676
      )
send_image_message6677=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6677.png',  #狂怒匕首6677
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6677.png'    #狂怒匕首6677
      )
send_image_message6691=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6691.png',  #暮色黑刃6691
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6691.png'    #暮色黑刃6691
      )
send_image_message6692=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6692.png',  #星蝕6692
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6692.png'    #星蝕6692
      )
send_image_message6693=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6693.png',  #潛行者之爪6693
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6693.png'    #潛行者之爪6693
      )
send_image_message6694=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6694.png',  #席利妲咒怨6694
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6694.png'    #席利妲咒怨6694
      )
send_image_message6695=ImageSendMessage(
      original_content_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6695.png',  #巨蛇鋒牙6695
      preview_image_url='https://ddragon.leagueoflegends.com/cdn/10.23.1/img/item/6695.png'    #巨蛇鋒牙6695
      )


In [ ]:
# Button篇 設定模板消息，指定其參數細節。   07功能   @實況主  會跳出大概5位實況主 按鍵模板提供選擇

#引入訊息事件、樣板發送訊息、回傳事件
from linebot.models import (
    MessageEvent, TemplateSendMessage , PostbackEvent
)

#引入按鍵模組
from linebot.models.template import(
    ButtonsTemplate
)

# alt_text: Line簡覽視窗所出現的說明文字
# template: 所使用的模板  使用了按鍵模板

# ButtonsTemplate: 按鍵模板
#     thumbnail_image_url: 展示圖片
#     title: 標題
#     text: 說明文字
#     actions: 模板行為所使用的行為  行為 :回傳動作、URI動作、網頁應用程式動作、日期時間選擇器動作
#     data: 觸發postback後用戶回傳值，可以對其做商業邏輯處理  等於是暗傳一個回資料庫的值



#按鍵範本訊息舉例
#ButtonTemplateSendMessage python Line example

buttons_template_message = TemplateSendMessage(
    alt_text='按鍵範本說明文字',
    template=ButtonsTemplate(
        title='實況主',
        text='下面有一些實況主可以選擇!',
        actions=[
        { 
        "type": "uri",
        "label": "NeVeR_LoEs的實況",
        "uri": "https://www.twitch.tv/never_loses"
        
        },
        { 
        "type": "uri",
        "label": "BeBe的實況",
        "uri": "https://www.twitch.tv/bebelolz"
        

        },
        { 
        "type": "uri",
        "label": "史丹利的實況",
        "uri": "https://www.twitch.tv/relaxing234"
        
        },
        { 
        "type": "uri",
        "label": "丁特的YT",
        "uri": "https://www.youtube.com/user/Dinterlolz"
        
        }

              ],
                 )
                          )  


In [ ]:
#@攻略的按鍵範本
# Button篇 設定模板消息，指定其參數細節。   07功能   @實況主  會跳出大概5位實況主 按鍵模板提供選擇

#引入訊息事件、樣板發送訊息、回傳事件
from linebot.models import (
    MessageEvent, TemplateSendMessage , PostbackEvent
)

#引入按鍵模組
from linebot.models.template import(
    ButtonsTemplate
)

# alt_text: Line簡覽視窗所出現的說明文字
# template: 所使用的模板  使用了按鍵模板

# ButtonsTemplate: 按鍵模板
#     thumbnail_image_url: 展示圖片
#     title: 標題
#     text: 說明文字
#     actions: 模板行為所使用的行為  行為 :回傳動作、URI動作、網頁應用程式動作、日期時間選擇器動作
#     data: 觸發postback後用戶回傳值，可以對其做商業邏輯處理  等於是暗傳一個回資料庫的值



#按鍵範本訊息舉例
#ButtonTemplateSendMessage python Line example

buttons_template_message1 = TemplateSendMessage(
    alt_text='按鍵範本說明文字',
    template=ButtonsTemplate(
        title='攻略網址',
        text='網址如下',
        actions=[
        { 
        "type": "uri",
        "label": "opgg",
        "uri": "https://tw.op.gg/champion/statistics"
        
        },
        { 
        "type": "uri",
        "label": "PROBUILDS",
        "uri": "https://www.probuilds.net/"
        

        }
       

              ],
                 )
                          )  


In [ ]:
#簡單聊天跟傳送貼圖  


#將訊息事件、貼圖傳送模組引入
from linebot.models import (
    MessageEvent 
)

from linebot.models import (
    StickerSendMessage
)

chat = StickerSendMessage(
    package_id='1',
    sticker_id='3'
)


In [ ]:
# 將訊息事件、文字訊息、寄送文字與圖片 模組引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)

# 設計一個字典，當用戶輸入相應文字消息時，系統會從此挑選消息  裝備字典   跟@實況主
# 根據自定義字典  設定什麼字傳什麼圖
template_message_dict = {  #寄送一張縮圖為鞋子點下去是鞋子的圖
  "@鞋子":send_image_message1001 , "@仙女護符":send_image_message1004 ,"@治療寶珠":send_image_message1006 ,
  "@巨人腰帶":send_image_message1011 ,"@靈巧披風":send_image_message1018 ,"@爆裂魔杖":send_image_message1026 ,
  "@藍水晶":send_image_message1027 , "@紅水晶":send_image_message1028 ,"@布甲":send_image_message1029 ,
  "@鎖子甲":send_image_message1031 , "@抗魔斗篷":send_image_message1033 ,"@餘燼之刃":send_image_message1035 ,
  "@長劍":send_image_message1036 ,"@十字鎬":send_image_message1037 ,"@暴風大劍":send_image_message1038 ,
  "@雹刃":send_image_message1039 ,"@短劍":send_image_message1042 ,"@反曲弓":send_image_message1043 ,
  "@增幅典籍":send_image_message1052 ,"@吸血鬼權杖":send_image_message1053 ,"@多蘭之盾":send_image_message1054 ,
  "@多蘭之劍":send_image_message1055 ,"@多蘭之戒":send_image_message1056 ,"@負極斗篷":send_image_message1057 ,
  "@巨型魔棒":send_image_message1058 ,"@黑暗玉璽":send_image_message1082 ,"@汰除品":send_image_message1083 ,
  "@生命藥水":send_image_message2003 ,"@科奇碎片":send_image_message2015 ,"@回復藥水":send_image_message2031 ,
  "@污濁藥劑":send_image_message2033 ,"@蘇瑞亞的戰歌":send_image_message2065 ,"@抗擊藥劑":send_image_message2138 ,
  "@巫魔藥劑":send_image_message2139 ,"@憤怒藥劑":send_image_message2140 ,"@預備鐘錶":send_image_message2419 ,
  "@停滯鐘錶":send_image_message2420 ,"@輕魔之靴":send_image_message2422 ,"@完美校準的停滯鐘錶":send_image_message2423 ,
  "@深淵面具":send_image_message3001 ,"@大天使之杖":send_image_message3003 ,"@魔劍正宗":send_image_message3004 ,
  "@狂戰士護脛":send_image_message3006 ,"@輕靈之靴":send_image_message3009 ,"@生化科技淨化器":send_image_message3011 ,
  "@法師之靴":send_image_message3020 ,"@冰川圓盾":send_image_message3024 ,"@守護天使":send_image_message3026 ,
  "@無盡之刃":send_image_message3031 ,"@致死宣告":send_image_message3033 ,"@最後耳語":send_image_message3035 ,
  "@多明尼克的問候":send_image_message3036 ,"@熾天使的擁抱":send_image_message3040 ,"@靈魂竊取者":send_image_message3041 ,
  "@終極魔劍":send_image_message3042 ,"@淨蝕之槌":send_image_message3044 ,"@幻影之舞":send_image_message3046 ,
  "@鍍板鋼蓋":send_image_message3047 ,"@錫柯的聚合之力":send_image_message3050 ,"@爐縛之斧":send_image_message3051,
  "@史特拉克手套":send_image_message3053 ,"@閃耀劍":send_image_message3057 ,"@振奮盔甲":send_image_message3065 ,
  "@飛翼月甲":send_image_message3066 ,"@燃燒寶石":send_image_message3067 ,"@日炎聖盾":send_image_message3068 ,
  "@女神之淚":send_image_message3070 ,"@黑色切割者":send_image_message3071 ,"@嗜血者":send_image_message3072 ,
  "@狂怒九頭蛇":send_image_message3074 ,"@荊棘之甲":send_image_message3075 ,"@刺藤胸甲":send_image_message3076 ,
  "@海神之斧":send_image_message3077 ,"@三相之力":send_image_message3078 ,"@守望者鎧甲":send_image_message3082 ,
  "@好戰者鎧甲":send_image_message3083 ,"@芮蘭颶風箭":send_image_message3085 ,"@狂熱雙刀":send_image_message3086 ,
  "@死亡之帽":send_image_message3089 ,"@智慧末刃":send_image_message3091 ,"@衝擊火砲":send_image_message3094 ,
  "@狂暴利刃":send_image_message3095 ,"@巫妖之禍":send_image_message3100 , "@女妖面紗":send_image_message3102 ,
  "@軍團聖盾":send_image_message3105 ,"@贖罪神石":send_image_message3107 ,"@惡魔法典":send_image_message3108 ,
  "@騎士誓願":send_image_message3109 ,"@冰霜之心":send_image_message3110 ,"@水星之靴":send_image_message3111 ,
  "@蒼穹神火":send_image_message3113 ,"@禁忌之像":send_image_message3114 ,"@納什之牙":send_image_message3115 ,
  "@瑞萊的冰晶節杖":send_image_message3116 ,"@疾行之靴":send_image_message3117 ,"@死刑宣告":send_image_message3123 ,
  "@鬼索的狂暴之刃":send_image_message3124 ,"@考菲爾德的戰槌":send_image_message3133 ,"@虛空之杖":send_image_message3135 ,
  "@水星彎刀":send_image_message3139 ,"@水銀飾帶":send_image_message3140 ,  "@妖夢鬼刀":send_image_message3142 ,
  "@蘭頓之兆":send_image_message3143 ,"@海克斯科技發電機":send_image_message3145 ,"@海克斯科技火箭腰帶":send_image_message3152 ,
  "@殞落王者之劍":send_image_message3153 ,"@吸魔劍":send_image_message3155 ,"@魔提斯深淵":send_image_message3156 ,
  "@中婭沙漏":send_image_message3157 ,"@愛歐尼亞之靴":send_image_message3158 ,"@黑魔禁書":send_image_message3165 ,
  "@守護者之刃":send_image_message3177 ,"@闇影戰戟":send_image_message3179 ,"@愉悅刀刃":send_image_message3181 ,
  "@日輪的加冕":send_image_message3190 ,"@探索者護腕":send_image_message3191 ,"@石像鬼磐核":send_image_message3193 ,
  "@亡靈斗篷":send_image_message3211 ,"@米凱的祝福":send_image_message3222 ,"@熾灼魔器":send_image_message3504 ,
  "@奪魄之鐮":send_image_message3508 ,"@死者肩甲":send_image_message3742 ,"@泰坦九頭蛇":send_image_message3748 ,
  "@水晶護腕":send_image_message3801 ,"@失落之章":send_image_message3802 ,"@夜色緣界":send_image_message3814 ,
  "@盜竊之刃":send_image_message3850 ,"@霜牙":send_image_message3851 ,"@真霜碎片":send_image_message3853 ,
  "@鋼鐵護肩":send_image_message3854 ,"@符文鋼肩":send_image_message3855 ,"@白石韌肩":send_image_message3857 ,
  "@遺跡之盾":send_image_message3858 ,"@巨石峰圓盾":send_image_message3859 ,"@群山堡壘":send_image_message3860 ,
  "@幽魂之鐮":send_image_message3862 ,"@駭懼之鐮":send_image_message3863 ,"@黯霧之鐮":send_image_message3864 ,
  "@遺忘寶珠":send_image_message3916 ,"@帝王命令":send_image_message4005 ,"@自然之力":send_image_message4401 ,
  "@視界專注":send_image_message4628 ,"@星河馳騁":send_image_message4629 ,"@枯萎寶石":send_image_message4630 ,
  "@蒼翠屏障":send_image_message4632 ,"@峽谷製造者":send_image_message4633 ,"@榨血睥睨":send_image_message4635 ,
  "@闇夜收割者":send_image_message4636 ,"@惡魔之擁":send_image_message4637 ,"@警戒視石":send_image_message4638 ,
  "@活耀視石":send_image_message4641 ,"@班德爾玻璃之鏡":send_image_message4642 ,"@警覺視石":send_image_message4643 ,
  "@鐵刺之鞭":send_image_message6029 ,"@銀織黎明":send_image_message6035 ,"@死亡之舞":send_image_message6333 ,
  "@生化龐克鏈鋸之劍":send_image_message6609 ,"@流水之杖":send_image_message6616 ,"@月之石再生裝置":send_image_message6617 ,
  "@飲血戰錘":send_image_message6630 ,"@步履破壞者":send_image_message6631 ,"@聖裂之杖":send_image_message6632 ,
  "@黎安卓的苦楚":send_image_message6653 ,"@盧登驟雨":send_image_message6655 ,"@永寒之霜":send_image_message6656 ,
  "@巴米灰燼":send_image_message6660 ,"@霜火霸拳":send_image_message6662 ,"@渦輪煉金罐":send_image_message6664 ,
  "@正午之顫":send_image_message6670 ,"@狂風之力":send_image_message6671 ,"@海妖殺手":send_image_message6672 ,
  "@不朽盾弓":send_image_message6673 ,"@那歐疾刃":send_image_message6675 ,"@蒐集者":send_image_message6676 ,
  "@狂怒匕首":send_image_message6677 ,"@暮色黑刃":send_image_message6691 ,"@星蝕":send_image_message6692 ,
  "@潛行者之爪":send_image_message6693 ,"@席利妲咒怨":send_image_message6694 ,"@巨蛇鋒牙":send_image_message6695 ,
  "@實況主":buttons_template_message,  #按鍵範本訊息 實況主按鍵範本
  "@攻略":buttons_template_message1, "@安安":chat
}

In [ ]:
# handler如果你收到MessageEvent ，且為TextMessage，做下面的方法
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
#判斷  event 裡面的message ，裡面text是否有包含@

    # prinnt((event.message.text))
    
    if(event.message.text.find('@')!= -1):  #有加@ 拿字典裡的文字 template_message_dict.get(event.message.text)
        line_bot_api.reply_message(
        event.reply_token,
        template_message_dict.get(event.message.text)
        )

    else:
        line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text="字典內查無此字，請輸入@裝備名或@實況主或@攻略跟@安安")
        )
    

In [ ]:
#用戶點擊button後，觸發postback event，對其回傳做相對應處理  07功能

@handler.add(PostbackEvent)
def handle_post_message(event):
    user_profile = line_bot_api.get_profile(event.source.user_id)
    if (event.postback.data.find('Data1')== 0):
        with open("user_profile_business.txt", "a") as myfile:
            myfile.write(json.dumps(vars(user_profile),sort_keys=True))
            myfile.write('\n')
            line_bot_api.reply_message(
            event.reply_token,
                TextMessage(
                    text='這是NL的實況'
                )
            )

    elif (event.postback.data.find('Data2') == 0):
        with open("user_profile_tutorial.txt", "a") as myfile:
            myfile.write(json.dumps(vars(user_profile),sort_keys=True))
            myfile.write('\n')
            line_bot_api.reply_message(
            event.reply_token,
                TextMessage(
                    text='這是BeBe的實況'
                )
            )
    elif (event.postback.data.find('Data3') == 0):
        with open("user_profile_tutorial.txt", "a") as myfile:
            myfile.write(json.dumps(vars(user_profile),sort_keys=True))
            myfile.write('\n')
            line_bot_api.reply_message(
            event.reply_token,
                TextMessage(
                    text='史丹利的實況'
                )
            )
    elif (event.postback.data.find('Data4') == 0):
        with open("user_profile_tutorial.txt", "a") as myfile:
            myfile.write(json.dumps(vars(user_profile),sort_keys=True))
            myfile.write('\n')
            line_bot_api.reply_message(
            event.reply_token,
                TextMessage(
                    text='這是丁特的YT'
                )
            )
    elif (event.postback.data.find('Data5') == 0):
        with open("user_profile_tutorial.txt", "a") as myfile:
            myfile.write(json.dumps(vars(user_profile),sort_keys=True))
            myfile.write('\n')
            line_bot_api.reply_message(
            event.reply_token,
                TextMessage(
                    text='這是大丸的實況'
                )
            )
    else:
        pass


In [ ]:
# 運行主程序
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://ea9b9000d158.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
{"events":[{"type":"follow","replyToken":"4c57b1ba702344dcaee8aa23dd003c86","source":{"userId":"Ufabdb2f3f29f108c3db07f450ac8956d","type":"user"},"timestamp":1606909723073,"mode":"active"}],"destination":"U65f5b8aa4dbfc84d5d0ee8aea5cfe9ba"}


127.0.0.1 - - [02/Dec/2020 11:48:44] "POST / HTTP/1.1" 200 -


{"events":[{"type":"message","replyToken":"3c50ac66f47148ad96bed205108f9c75","source":{"userId":"Ufabdb2f3f29f108c3db07f450ac8956d","type":"user"},"timestamp":1606909727791,"mode":"active","message":{"type":"text","id":"13133148876097","text":"@攻略"}}],"destination":"U65f5b8aa4dbfc84d5d0ee8aea5cfe9ba"}


...

127.0.0.1 - - [02/Dec/2020 11:48:52] "POST / HTTP/1.1" 200 -


{"events":[{"type":"message","replyToken":"00ccb88a5af6489781e3c9d0bfd0e2fb","source":{"userId":"Ufabdb2f3f29f108c3db07f450ac8956d","type":"user"},"timestamp":1606909734320,"mode":"active","message":{"type":"text","id":"13133149447295","text":"@安安"}}],"destination":"U65f5b8aa4dbfc84d5d0ee8aea5cfe9ba"}


127.0.0.1 - - [02/Dec/2020 11:48:55] "POST / HTTP/1.1" 200 -


{"events":[{"type":"message","replyToken":"939a825e88b54dfb8d690742e219b124","source":{"userId":"Ufabdb2f3f29f108c3db07f450ac8956d","type":"user"},"timestamp":1606909745460,"mode":"active","message":{"type":"text","id":"13133150442600","text":"@鞋子"}}],"destination":"U65f5b8aa4dbfc84d5d0ee8aea5cfe9ba"}


127.0.0.1 - - [02/Dec/2020 11:49:06] "POST / HTTP/1.1" 200 -
